## HiFiGan Results

In [1]:
import os
import json
from pathlib import Path

import wandb

import torch
import pandas as pd
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt

from nemo.collections.tts.models import FastPitchModel, HifiGanModel

[NeMo W 2022-12-08 17:17:53 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-12-08 17:17:53 experimental:27] Module <class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-12-08 17:17:53 experimental:27] Module <class 'nemo.collections.tts.models.radtts.RadTTSModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [12]:
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel

Let's also download the pretrained checkpoint that we want to finetune from. NeMo will save checkpoints to `~/.cache`, so let's move that to our current directory. 

*Note: please, check that `home_path` refers to your home folder. Otherwise, change it manually.*

In [2]:
SPEAKER_ID = 9017

WANDB_PROJECT = "tts-workshop"
WANDB_ENTITY = "capecape" # replace with your wandb username or team

In [3]:
# which split we are using
validation_split_artifact = f'{WANDB_ENTITY}/{WANDB_PROJECT}/9017_5_split:v0'

# which model
fastpitch_artifact = f'{WANDB_ENTITY}/{WANDB_PROJECT}/model-2022-12-07_17-56-27:v2'
hifigan_artifact   = f'{WANDB_ENTITY}/{WANDB_PROJECT}/model-32pmu5t7:v3'

In [4]:
wandb.init(entity=WANDB_ENTITY, project=WANDB_PROJECT, job_type="final_validation")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


We can grab the fine tuned models from the `wandb` artifact:

In [5]:
split_artifact = wandb.use_artifact(validation_split_artifact)
split_artifact_dir = split_artifact.download()

fastpitch_artifact = wandb.use_artifact(fastpitch_artifact, type='model')
fastpitch_artifact_dir = fastpitch_artifact.download()

hifigan_artifact = wandb.use_artifact(hifigan_artifact, type='model')
hifigan_artifact_dir = hifigan_artifact.download()

wandb:   2 of 2 files downloaded.  
wandb: Downloading large artifact model-2022-12-07_17-56-27:v2, 524.07MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact model-32pmu5t7:v3, 969.73MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [6]:
def ls(path): return list(Path(path).iterdir())

In [7]:
ls(split_artifact_dir)

[PosixPath('artifacts/9017_5_split:v0/9017_manifest_valid_local.json'),
 PosixPath('artifacts/9017_5_split:v0/9017_manifest_train_local.json')]

In [8]:
ls(fastpitch_artifact_dir)

[PosixPath('artifacts/model-2022-12-07_17-56-27:v2/model.ckpt')]

In [9]:
ls(hifigan_artifact_dir)

[PosixPath('artifacts/model-32pmu5t7:v3/model.ckpt')]

load FastPitch

In [10]:
last_ckpt = str(ls(fastpitch_artifact_dir)[0])
print(last_ckpt)

spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda();

artifacts/model-2022-12-07_17-56-27:v2/model.ckpt
[NeMo I 2022-12-08 17:18:03 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-12-08 17:18:34 experimental:27] Module <class 'nemo_text_processing.g2p.modules.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-12-08 17:18:35 modules:95] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-12-08 17:18:35 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: 9017_manifest_train_local.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n

[NeMo I 2022-12-08 17:18:35 features:267] PADDING: 1


In [11]:
last_ckpt = str(ls(hifigan_artifact_dir)[0])
print(last_ckpt)

vocoder_model = HifiGanModel.load_from_checkpoint(last_ckpt)
vocoder_model.eval().cuda();

artifacts/model-32pmu5t7:v3/model.ckpt


[NeMo W 2022-12-08 17:18:38 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: hifigan_train_ft.json
      sample_rate: 22050
      n_segments: 8192
      max_duration: null
      min_duration: 0.75
      load_precomputed_mel: true
      hop_length: 256
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 16
      num_workers: 4
      pin_memory: true
    
[NeMo W 2022-12-08 17:18:38 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
     

[NeMo I 2022-12-08 17:18:38 features:267] PADDING: 0
[NeMo I 2022-12-08 17:18:38 features:275] STFT using exact pad
[NeMo I 2022-12-08 17:18:38 features:267] PADDING: 0
[NeMo I 2022-12-08 17:18:38 features:275] STFT using exact pad


Let's log the model predictions to `W&B`

In [15]:
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

In [16]:
valid_df = pd.read_json(Path(split_artifact_dir)/f"{SPEAKER_ID}_manifest_valid_local.json", lines=True)
valid_df

,audio_filepath,text,duration,text_no_preprocessing,text_normalized
0,audio/dartagnan03part3_55_dumas_0101.wav,it is a pride natural to my race to pretend to...,6.28,it is a pride natural to my race to pretend to...,it is a pride natural to my race to pretend to...
1,audio/antoinetteromances4_25_dumas_0150.wav,ah if you had stayed in paris this would not h...,4.74,"Ah, if you had stayed in Paris this would not ...","Ah, if you had stayed in Paris this would not ..."


In [17]:
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

In [18]:
def generate_audio(text, speaker_id):
    "Generate MEL and Synth Audio"
    spec, audio = infer(spec_model, vocoder_model, text, speaker=speaker_id)
    return spec, audio.flatten()

In [19]:
new_speaker_id = SPEAKER_ID
duration_mins = 5
mixing = False
original_speaker_id = "ljspeech"

In [20]:
table = wandb.Table(columns=['Text', 'Real validation audio', f'HiFiGAN {new_speaker_id}', 'Spec'])

sample_rate=22050

for _, val_record in valid_df.iterrows():
    speaker_spec, speaker_audio = generate_audio(val_record['text'], speaker_id=new_speaker_id)
    row = [val_record["text_no_preprocessing"],
           wandb.Audio(val_record['audio_filepath'], sample_rate=sample_rate), 
           wandb.Audio(speaker_audio.flatten(), sample_rate=sample_rate),
           wandb.Image(speaker_spec)]
    table.add_data(*row)

wandb.log({"hifigan_predictions": table})

In [21]:
wandb.finish()